In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    output_dir: Path
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    warmup_steps: int
    weight_decay: float
    logging_steps: int
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    report_to: str
    eval_strategy: str
    predict_with_generate: bool
    generation_max_length: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:

        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            output_dir = params.output_dir,
            num_train_epochs= params.num_train_epochs,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            warmup_steps= params.warmup_steps,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps,
            report_to= params.report_to,
            eval_strategy= params.eval_strategy,
            predict_with_generate= params.predict_with_generate,
            generation_max_length= params.generation_max_length
        )

        return model_training_config

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import evaluate
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_from_disk

c:\Users\sahil\OneDrive\Desktop\Projects\AI Projects\Text-Summarizer-Application\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

    def evaluate(self, eval_pred):
        # Download the NLTK punctuation tokenizer and the required punkt_tab
        nltk.download("punkt")
        nltk.download("punkt_tab")

        # Load the ROUGE metric
        rouge_metric = evaluate.load("rouge")

        predictions, labels = eval_pred

        # Fix: Ensure predictions are within the valid tokenizer vocabulary range to avoid OverflowError
        vocab_size = len(self.tokenizer)
        predictions = np.clip(predictions, 0, vocab_size - 1)

        # 1. Decode the model's predictions back into text
        decoded_preds = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)

        # 2. Replace the -100s in the labels back to pad_token_id
        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        labels = np.clip(labels, 0, vocab_size - 1)
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        # 3. ROUGE-Lsum expects text to have newlines between sentences
        decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

        # 4. Calculate the ROUGE scores
        result = rouge_metric.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True,
            rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"]
        )

        # 5. Scale to percentages
        result = {key: value * 100 for key, value in result.items()}

        prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)

        return {k: round(v, 4) for k, v in result.items()}

    
    def train(self):
        seq2seq_data_collator = DataCollatorForSeq2Seq(self.tokenizer, model=self.model, label_pad_token_id=-100)
        
        training_args = Seq2SeqTrainingArguments(
            output_dir= self.config.output_dir,
            num_train_epochs= self.config.num_train_epochs,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            warmup_steps= self.config.warmup_steps,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            eval_steps= self.config.eval_steps,             
            save_steps= self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            report_to=self.config.report_to,
            eval_strategy=self.config.eval_strategy,
            predict_with_generate=self.config.predict_with_generate,
            generation_max_length=self.config.generation_max_length
        )

        tokenized_data = load_from_disk(self.config.data_path)

        trainer = Seq2SeqTrainer(
            model=self.model,
            args=training_args,
            processing_class=self.tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=tokenized_data["train"],
            eval_dataset=tokenized_data["validation"],
            compute_metrics= self.evaluate
        )

        trainer.train()

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_training_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2026-02-22 23:36:43,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-22 23:36:43,073: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-22 23:36:43,074: INFO: common: Directory created at: artifacts]
[2026-02-22 23:36:43,075: INFO: common: Directory created at: artifacts/model_trainer]


c:\Users\sahil\OneDrive\Desktop\Projects\AI Projects\Text-Summarizer-Application\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2,No log,2.807010,26.209300,7.739300,20.914600,23.753400,40.341100


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sahil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sahil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.

[2026-02-22 23:44:25,472: INFO: rouge_scorer: Using default tokenizer.]


KeyboardInterrupt: 